In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pickle
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator

In [2]:
def loadData():
    file = open('../input/vietnamese-foods-dataset/dataset/vietnamese_foods.data', 'rb')
    (images, labels) = pickle.load(file)
    file.close()
    print(images.shape)
    print(labels.shape)
    return images, labels

In [3]:
X, y = loadData()
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2)

In [4]:
images = ImageDataGenerator(rotation_range=20, zoom_range=0.15,
        rescale=1./255,
        width_shift_range=0.15,
        height_shift_range=0.15,
        horizontal_flip=True,
        brightness_range=[0.2,1.5], fill_mode="nearest")

In [5]:
# Create model using Sequential API
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(256,256,3)))

model.add(layers.Conv2D(32, (3,3), padding="same", activation='relu', kernel_initializer='he_uniform'))
model.add(layers.Conv2D(32, (3,3), padding="same", activation='relu', kernel_initializer='he_uniform'))
model.add(layers.MaxPool2D((2,2), strides=(2,2)))

model.add(layers.Conv2D(64, (3,3), padding="same", activation='relu', kernel_initializer='he_uniform'))
model.add(layers.Conv2D(64, (3,3), padding="same", activation='relu', kernel_initializer='he_uniform'))
model.add(layers.MaxPool2D((2,2), strides=(2,2)))

model.add(layers.Flatten())
model.add(layers.Dense(2048, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(2048, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(10))
model.add(layers.Softmax())

In [6]:
opt = SGD(learning_rate = 0.001, momentum = 0.9)
model.compile(optimizer = "adam", loss ='categorical_crossentropy', metrics = ['accuracy'])

In [7]:
# Training
history = model.fit(images.flow(X_train, y_train, batch_size=64), epochs=60,validation_data=images.flow(X_test, y_test, batch_size=64), verbose=1)
model.save("model.h5")

In [8]:
score = model.evaluate(X_test, y_test, verbose=2)
print("Loss = ", score[0])
print("Accuracy = ", score[1])